# Twitter Sentiment Analysis

This notebook aims to analyze Tweets made about the COVID-19 vaccine and determine whether the sentiment of the Tweets is negative, positive, or neutral.

## Libraries

In [1]:
import tweepy
import webbrowser
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('whitegrid')
import time

import config
pd.set_option('display.max_colwidth', None)

from tweepy import OAuthHandler
from tweepy import API

import os
from google.oauth2 import service_account
from google.cloud import translate_v2 as translate
from google.cloud import language


import nltk
from nltk.corpus import gutenberg, stopwords
from nltk import FreqDist
from nltk.tokenize import TweetTokenizer, word_tokenize
import string

import re
from textblob import TextBlob
from wordcloud import WordCloud

## Gather data from Twitter API

In [ ]:
# Developer API keys
# API keys in config.py in .gitignore
callback_uri = 'oob'

In [ ]:
auth = OAuthHandler(config.consumer_key, config.consumer_secret, callback_uri)
auth.set_access_token(config.access_token, config.access_token_secret)

In [ ]:
api = API(auth, wait_on_rate_limit= True, wait_on_rate_limit_notify= True)

In [ ]:
# show my Twitter handle
me = api.me()
print(me.screen_name)

In [ ]:
# simple query to get the hang of it
query = '#vaccine covid -RT -@ -http'
for i, status in enumerate(tweepy.Cursor(api.search, q= query).items(15)):
    print (i, status.text)

# filter only English tweets    
# query = '#vaccine covid'
# for i, status in enumerate(tweepy.Cursor(api.search, q= query, lang= 'en').items(50)):
#     print (i, status.text)

In [ ]:
''' This cell was run to collect the initial 5000 tweets and save them to a csv file
is commented out to prevent it from being run again


# collect 5000 tweets 
number_of_tweets = 5000
tweets = []
language = []
time = []

# only collect full text, language, and time it was posted
# remove any retweets '-RT'
for i in tweepy.Cursor(api.search, q= 'vaccine covid -RT',  
                       tweet_mode = 'extended').items(number_of_tweets):
    tweets.append(i.full_text)
    language.append(i.lang)
    time.append(i.created_at)

# turn tweets / lists into dataframe
df = pd.DataFrame({'time': time, 'language': language, 'tweets': tweets})

# save df to csv
df.to_csv('data/tweets.csv', index= False)
'''

Collect more tweets after waiting to avoid collecting the same tweets.

In [ ]:
# create function to get more tweets and add to existing csv
def get_more_tweets(num_tweets):
    number_of_tweets = num_tweets
    tweets = []
    language = []
    time = []

    # only collect full text, language, and time it was posted
    # remove any retweets '-RT'
    for i in tweepy.Cursor(api.search, q= 'vaccine covid -RT',  
                           tweet_mode = 'extended').items(number_of_tweets):
        tweets.append(i.full_text)
        language.append(i.lang)
        time.append(i.created_at)
        
    # turn tweets / lists into dataframe
    df = pd.DataFrame({'time': time, 'language': language, 'tweets': tweets})
    
    # add to existing csv file
    df.to_csv('data/tweets.csv', mode= 'a', index= False, header= False)

In [ ]:
# get tweets and save to csv through function

# n_tweets = 20
# get_more_tweets(n_tweets)

### Search for Tweets from 30 Days Ago

In [ ]:
# function to turn results into DataFrame
def extract_text_as_df(text_list):
    columns = set()
    allowed_types = [str, int]
    tweets_data = []
    for status in text_list:
        status_dict = dict(vars(status))
        keys = status_dict.keys()
        single_tweet_data = {"author": status.author.screen_name}
        for k in keys:
            try:
                v_type = type(status_dict[k])
            except:
                v_type = None
            if v_type != None:
                if v_type in allowed_types:
                    single_tweet_data[k] = status_dict[k]
                    columns.add(k)
        tweets_data.append(single_tweet_data)


    header_cols = list(columns)
    header_cols.append('author')
    df = pd.DataFrame(tweets_data, columns=header_cols)
    return df

In [ ]:
query = 'vaccine covid -RT'
results_30 = api.search_30_day('datacollection', query, maxResults= 100)

In [ ]:
old_results_df = extract_text_as_df(results_30)

In [ ]:
old_results_df

## Data Cleaning & EDA

In [ ]:
df = pd.read_csv('data/tweets.csv')

In [ ]:
df.info()

### Check and Remove Duplicates

In [ ]:
# check for duplicated tweets
duped = df[df.duplicated(subset= 'tweets')]
duped.info()

In [ ]:
# remove duplicates
df.drop_duplicates(subset= 'tweets', keep= 'first', inplace= True)
df.reset_index(drop= True, inplace= True)

# make sure they are gone
df.tweets.duplicated().sum()

In [ ]:
df.head(15)

### Looking at Languages

In [ ]:
lang_df = df.sort_values('language')

In [ ]:
plt.figure(figsize= (12, 8))
lang_df.language.value_counts(normalize=True).plot(kind= 'barh')
plt.title('Proportion of Languages Collected')
plt.xlim(-.01, 1)
plt.show()

In [ ]:
lang_df.language.value_counts(normalize=True)

In [ ]:
# what are these languages? get table from url
URL = 'https://developer.twitter.com/en/docs/twitter-for-websites/supported-languages'
tables = pd.read_html(URL)
print("There are : ",len(tables)," tables")
print(tables[0])

lang_table = tables[0]

In [ ]:
# format table
header_row = 0
lang_table.columns = lang_table.iloc[header_row]
lang_table = lang_table.drop(header_row)
lang_table.reset_index(drop= True, inplace= True)

lang_table.replace('English (default)', 'English', inplace= True)
# lang_table

In [ ]:
# save as csv in data folder
lang_table.to_csv('data/languages.csv', index= False)

In [ ]:
# look at 'und' language 
df[df['language'] == 'und']

It appears that the 'und' or possibly 'undetermined' language are tweets that do not include text to help us with our sentiment analysis. We will remove those from our DataFrame.

In [ ]:
df = df[df['language'] != 'und']
# df.info()

Merge the data with the languages to see the whole name of languages

In [ ]:
# merge data with language names
df2 = df.merge(lang_table, how= 'inner', left_on= 'language', right_on= 'Language code')
df2.info()

In [ ]:
# drop extra columns and rename them and change dates to datetime and drop time
df2.drop(columns = ['language', 'Language code'], inplace= True)
df2.rename(columns={'Name': 'language'}, inplace= True)
df2['time'] = pd.to_datetime(df2['time']).dt.date
# df2

Let's see how many Tweets are in each language?

In [ ]:
sns.countplot(y= 'language', data= df2)

The majority of Tweets are in English. Let's see what the other languages are.

In [ ]:
# countplot that excludes English to see detail
plt.figure(figsize= (10, 8))
sns.countplot(y= 'language', data= df2, palette='plasma',
              order= df2.language.value_counts().iloc[1:].index)
plt.title('Count of Tweets, Excluding English', fontsize= 14)
plt.xlabel('# of Tweets')
plt.ylabel('Language')
plt.show()

After English, the top 3 languages for Tweets about the COVID-19 vaccine are in French, Spanish, and Japanese.

In [ ]:
df2.head(10)

#### Translate to English using Google Cloud Platform's API

**Most of this section has been commented out to prevent running on Google Cloud.**

In [ ]:
# send in credentials that are saved on local computer
credentials = service_account.Credentials.from_service_account_file('../GCP_service_account.json')
translate_client = translate.Client(credentials = credentials)

In [ ]:
# see all languages that Google can translate
print(translate_client.get_languages())

In [ ]:
# separate non-English tweets for faster translating
part_df = df2.loc[16100:]
# part_df.language.value_counts(normalize= True)

In [ ]:
# create dictionary to save original tweet and translated one
# commented out to prevent re-running on Google Cloud
'''
results = {}

# translate tweets into English
for i in part_df.tweets.loc[:]:
    result = translate_client.translate(i, target_language= 'en')
    results[i] = result['translatedText']
'''

In [ ]:
# replace with English translation
# df2.tweets.replace(results, inplace= True)

In [ ]:
# preview previously French Tweets
# df2[df2['language'] == 'French']

In [ ]:
# save to csv file
# df2.to_csv('data/tweets_translated.csv', index= False)

## Text Sentiment
---
Does the tweet have a positive, negative, or neutral tone?

### Feature Engineering for Text Data
--- 
stopword removal
frequency distributions
lemmatization
bigramsm n-grams, and mutal information score

In [2]:
# load data from csv
df2 = pd.read_csv('data/tweets_translated.csv')

In [3]:
# remove language column
sentiment_df = df2[['time', 'tweets']]
sentiment_df.head()

### Google Cloud Platform Natural Language Processing

In [4]:
# input credentials from local file to access Google Cloud Platform
credentials = service_account.Credentials.from_service_account_file('../GCP_service_account.json')
nlp_client = language.Client(credentials = credentials)

In [6]:
# function to collect sentiment from Google Cloud
scores = []
magnitudes = []

def getGoogleSentiments(txt):
    document = nlp_client.document_from_text(txt)
    
    sent_analysis = document.analyze_sentiment()
    sentiment = sent_analysis.sentiment
    
    # get score and magnitude and add to list
    score = sentiment.score
    scores.append(score)
    magnitude = sentiment.magnitude
    magnitudes.append(magnitude)
    # add delay to prevent hitting rate limit
    time.sleep(5)


## Running here

In [ ]:
# get sentiment score and magnitude from function
google_sent = sentiment_df.tweets.apply(getGoogleSentiments)

_Google Cloud's Natural Language sentiment analysis markers:_

**Score** of the sentiment ranges between -1.0 (negative) and 1.0 (positive) and corresponds to the overall emotional leaning of the text.

**Magnitude** indicates the overall strength of emotion (both positive and negative) within the given text, between 0.0 and +inf. Unlike score, magnitude is not normalized; each expression of emotion within the text (both positive and negative) contributes to the text's magnitude (so longer text blocks may have greater magnitudes).

In [ ]:
# add lists to DataFrame as new columns
sentiment_df['Google Score'] = scores
sentiment_df['Google Magnitude'] = magnitudes

In [ ]:
# function to determine if Tweet was positive, negative, or neutral
def getTextAnalysis(p):
    if p < 0:
        return 'Negative'
    elif p == 0:
        return 'Neutral'
    else:
        return 'Positive'

In [ ]:
sentiment_df['Google Sentiment'] = sentiment_df['Google Score'].apply(getTextAnalysis)

In [ ]:
# plot as bar graph
plt.figure(figsize= (10, 6))
sns.countplot(data= sentiment_df, x = 'Google Sentiment')
plt.title('Sentiment Analysis of Tweets about COVID-19 Vaccine', fontsize = 14)
plt.xlabel('Score')
plt.show()

In [ ]:
# plot polarity and subjectivity and sentiment of the Tweet
plt.figure(figsize= (10,8))
sns.scatterplot(x= 'Google Score', y= 'Google Magnitude', data= sentiment_df, hue= 'Google Sentiment')
plt.title('Polarity and Subjectivity of COVID-19 Vaccine Tweets by Sentiment', 
          fontsize = 14, fontweight= 'bold')
plt.show()

In [ ]:
# save to csv file
# sentiment_df.to_csv('data/tweets_google_sentiment.csv', index= False)

In [ ]:
sent1 = sentiment_df[:500]

In [ ]:
google_sent = sent1.tweets.apply(getGoogleSentiments)

In [ ]:
sent1

In [ ]:
def getMoreGoogleSentiments(txt):
    document = nlp_client.document_from_text(txt)
    
    sent_analysis = document.analyze_sentiment()
    sentiment = sent_analysis.sentiment
    
    score = sentiment.score
    scores.extendnd(score)
    magnitude = sentiment.magnitude
    magnitudes.extend(magnitude)

In [ ]:
sent2 = sentiment_df[500:1000]

In [ ]:
google_sent = sent2.tweets.apply(getMoreGoogleSentiments)

### Sentiment Analysis with TextBlob

In [ ]:
sentiment_df = pd.read_csv('data/tweets_google_sentiment.csv')

In [ ]:
# functions to clean tweets, get subjectivity and polarity scores and sentiment
def cleanTweet(txt):
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|(amp)", 
                           " ", txt).split())

def getTextSubjectivity(txt):
    '''Subjective sentences generally refer to personal opinion, emotion 
    or judgment whereas objective refers to factual information. 
    Subjectivity is also a float which lies in the range of [0,1].'''
    return TextBlob(txt).sentiment.subjectivity

def getTextPolarity(txt):
    '''Polarity is float which lies in the range of [-1,1] 
    where 1 means positive statement and -1 means a negative statement.'''
    return TextBlob(txt).sentiment.polarity

In [ ]:
# apply functions to tweets
sentiment_df['tweets'] = sentiment_df['tweets'].apply(clean_tweet)
sentiment_df['TB Subjectivity'] = sentiment_df['tweets'].apply(getTextSubjectivity)
sentiment_df['TB Polarity'] = sentiment_df['tweets'].apply(getTextPolarity)
sentiment_df['TB Sentiment'] = sentiment_df['TB Polarity'].apply(getTextAnalysis)
sentiment_df

In [ ]:
# plot as bar graph
plt.figure(figsize= (10, 6))
sns.countplot(data= sentiment_df, x = 'TB Sentiment')
plt.title('Sentiment Analysis of Tweets about COVID-19 Vaccine', fontsize = 14)
plt.xlabel('Score')
plt.show()

In [ ]:
# plot polarity and subjectivity and sentiment of the Tweet
plt.figure(figsize= (10,8))
sns.scatterplot(x= 'TB Polarity', y= 'TB Subjectivity', data= sentiment_df, hue= 'TB Sentiment')
plt.title('Polarity and Subjectivity of COVID-19 Vaccine Tweets by Sentiment', 
          fontsize = 14, fontweight= 'bold')
plt.show()

In [ ]:
# save all to csv
# sentiment_df.to_csv('data/tweets_sentiment_data.csv', index= False)

### Word Cloud, Word Freqency and Bigram

In [ ]:
# create word cloud
words = ' '.join([tweet for tweet in sentiment_df['tweets']])
wc = WordCloud(background_color= '#222222').generate(words)
plt.figure(figsize= (10, 6))
plt.imshow(wc.recolor(colormap= 'spring'))
plt.axis('off')
plt.title('COVID-19 Vaccine Word Cloud')
plt.show()

In [ ]:
# tokenizing
tokens = dataf['tweets'].map(word_tokenize)
tokens

In [ ]:
 # remove stop words make make all words lowercase
stopped_data = [w.lower() for l in tokens for w in l if w not in stopwords_list]
print(len(stopped_data))
stopped_data

In [ ]:
# show 20 most common words
words_freq = FreqDist(stopped_data)
freq_words_20 = words_freq.most_common(20)

In [ ]:
# plot 20 most frequent words


In [ ]:
#bigrams
bigram_measures = nltk.collocations.BigramAssocMeasures()

tweets_finder = nltk.collocations.BigramCollocationFinder.from_words(stopped_data)

tweets_scored = tweets_finder.score_ngrams(bigram_measures.raw_freq)
tweets_scored[:10]

### Differences from Google and TextBlob

## Classification Modeling
---
To build models of our tweets, we first need to do some more cleaning that Google and TextBlob didn't require.

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

from nltk import word_tokenize, TweetTokenizer

from nltk.tokenize import TweetTokenizer
import re 
from textblob import TextBlob
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)
import string

stopwords_list = stopwords.words('english')
stopwords_list += list(string.punctuation)